In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import numpy as np
from six import BytesIO
from PIL import Image

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import config_util
from object_detection.builders import model_builder

In [ ]:
%matplotlib inline

We are instantiating a model to make use of its pre-process function for images (e.g. resizing, etc)

In [ ]:
pipeline_config = '/home/ubuntu/balloon_project/models/ssd_mobilenet_v2_320x320_coco17_tpu-8/run1/pipeline.config'
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
model_config.ssd.num_classes = 1
model_config.ssd.freeze_batchnorm = True
detection_model = model_builder.build(
      model_config=model_config, is_training=True)

In [ ]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
        path: a file path.

    Returns:
        uint8 numpy array with shape (sample, img_height, img_width, 3)
    """
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (1,im_height, im_width, 3)).astype(np.uint8)
    
def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_name=None):
    """Wrapper function to visualize detections.

    Args:
    image_np: uint8 numpy array with shape (img_height, img_width, 3)
    boxes: a numpy array of shape [N, 4]
    classes: a numpy array of shape [N]. Note that class indices are 1-based,
      and match the keys in the label map.
    scores: a numpy array of shape [N] or None.  If scores=None, then
      this function assumes that the boxes to be plotted are groundtruth
      boxes and plot all boxes as black with no classes or scores.
    category_index: a dict containing category dictionaries (each holding
      category index `id` and category name `name`) keyed by category indices.
    figsize: size for the figure.
    image_name: a name for the image file.
    """
    image_np_with_annotations = image_np.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_annotations,
        boxes,
        classes,
        scores,
        category_index,
        use_normalized_coordinates=True,
        min_score_thresh=0.5)
    plt.figure(figsize=figsize)
    if image_name:
        plt.imsave(image_name, image_np_with_annotations)
    else:
        plt.imshow(image_np_with_annotations)
        
def detect(interpreter, input_tensor):
    """Run detection on an input image.

    Args:
        interpreter: tf.lite.Interpreter
        input_tensor: A [1, height, width, 3] Tensor of type tf.float32.
          Note that height and width can be anything since the image will be
          immediately resized according to the needs of the model within this
          function.

    Returns:
        A dict containing 3 Tensors (`detection_boxes`, `detection_classes`,
          and `detection_scores`).
    """
    
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # We use the original model for pre-processing, since the TFLite model doesn't
    # include pre-processing.
    preprocessed_image, shapes = detection_model.preprocess(input_tensor)
    interpreter.set_tensor(input_details[0]['index'], preprocessed_image.numpy())
    interpreter.invoke()

    scores = interpreter.get_tensor(output_details[0]['index'])
    boxes = interpreter.get_tensor(output_details[1]['index'])
    count = interpreter.get_tensor(output_details[2]['index'])
    classes = interpreter.get_tensor(output_details[3]['index'])
    
    return boxes, classes, scores

In [ ]:
test_image_path = 'test_samples/sample_balloon.jpeg'
test_image_np = load_image_into_numpy_array(test_image_path)

tflite_model_path = 'exported_models/ssd_mobilenet_v2_320x320_coco17_tpu-8/tflite/model.tflite'
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

label_id_offset = 1
input_tensor = tf.convert_to_tensor(test_image_np, dtype=tf.float32)
boxes, classes, scores = detect(interpreter, input_tensor)

PATH_TO_LABELS = 'data/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

plot_detections(
    test_image_np[0],
    boxes[0],
    classes[0].astype(np.uint32) + label_id_offset,
    scores[0],
    category_index, figsize=(15, 20))
